<a href="https://colab.research.google.com/github/illaho/INVEST_IO/blob/master/investing_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!pip install webdriver-manager
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 911kB 5.3MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [95.7 kB]
Get:11 http://archive.ubuntu.com/ubuntu bioni

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np
import json
import pprint
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver', options=options)

wd.get('https://www.investing.com/')
# Click on 'I agree' button and wait for a second
#wd.find_element_by_xpath("//button[@value='agree']").click()
#time.sleep(1)

In [ ]:
print(wd.title)
wd.save_screenshot('screenshot.png')

%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('/content/screenshot.png')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
stocks = ["apple-computer-inc",
         "amazon-com-inc", 
         "microsoft-corp", 
         "google-inc", 
         "facebook-inc"]
         
detail_news = []

for stock in tqdm(stocks):
  for num in tqdm(range(1,208)):
      wd.get(f"https://www.investing.com/equities/{stock}-news/{num}")
      time.sleep(2 + np.random.randn()*0.4)
      wait = WebDriverWait(wd, 30)
      news_list = wd.find_elements_by_css_selector('#leftColumn > div.mediumTitle1 > article > div.textDiv')
      print(len(news_list))
      for news in news_list:
        author = wd.find_elements_by_css_selector('#leftColumn div.mediumTitle1 article div.textDiv span')[0].text.split("By")[-1].split("-")[0]
        #leftColumn > div.mediumTitle1 > article:nth-child(1) > div.textDiv > div > span:nth-child(1) span.articleDetails
        container_id = news.find_element_by_css_selector("a").get_attribute("href")
        if "-" in container_id:
            news_id = container_id.split("-")[-1]
        else: 
            news_id = container_id.split("/")[-1]
        ticker = wd.find_element_by_css_selector('#leftColumn > div.instrumentHeader > h2').text.split(" ")[0]
        url = news.find_element_by_css_selector("a").get_attribute("href")
        title = news.find_element_by_css_selector("a").text
        if len(news_id) > 5:
          detail_news.append({'news_id': news_id,
                              'ticker': ticker,
                              'author': author,
                              'title': title,
                              'url': url,
                              })
print(len(detail_news))
pprint.pprint(detail_news)

In [ ]:
ds_detail_news = pd.DataFrame(detail_news)
ds_detail_news.set_index("news_id")
ds_detail_news.head()
ds_detail_news.tail()

,news_id,ticker,author,title,url
9005,2049753,FB,Reuters,'Do Not Sell My Info': U.S. retailers rush to ...,https://www.investing.com/news/technology-news...
9006,2049785,FB,Reuters,Brazil fines Facebook $1.6 million for imprope...,https://www.investing.com/news/technology-news...
9007,2050046,FB,Reuters,Gunman in Texas church had trained worshippers...,https://www.investing.com/news/world-news/gunm...
9008,2049623,FB,Reuters,"Hong Kong to end year with multiple protests, ...",https://www.investing.com/news/world-news/hong...
9009,2049706,FB,Reuters,Political exercises: Thai rivals plan to mobil...,https://www.investing.com/news/world-news/poli...


In [ ]:
ds_detail_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9010 entries, 0 to 9009
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   news_id  9010 non-null   object
 1   ticker   9010 non-null   object
 2   author   9010 non-null   object
 3   title    9010 non-null   object
 4   url      9010 non-null   object
dtypes: object(5)
memory usage: 352.1+ KB


In [ ]:
ds_detail_news.to_csv('/ds_news_investing.csv')

In [ ]:
ds_detail_news = pd.read_csv('/content/ds_news_investing.csv')
ds_detail_news.loc[7179:]

,Unnamed: 0,news_id,ticker,author,title,url
7179,7179,2287405,FB,Reuters,Hong Kong police fire pepper balls at proteste...,https://www.investing.com/news/world-news/hund...
7180,7180,2287183,FB,Reuters,"Weekly Comic: Apple, Tesla Slide Spark Fears O...",https://www.investing.com/news/economy/weekly-...
7181,7181,2287375,FB,Reuters,Philippine police draw flak for plan to monito...,https://www.investing.com/news/technology-news...
7182,7182,2287350,FB,Reuters,Several boats sink at Trump parade on Texas lake,https://www.investing.com/news/politics/severa...
7183,7183,2287160,FB,Reuters,Hawaii team backed by former All Blacks pulls ...,https://www.investing.com/news/general/hawaii-...
...,...,...,...,...,...,...
9005,9005,2049753,FB,Reuters,'Do Not Sell My Info': U.S. retailers rush to ...,https://www.investing.com/news/technology-news...
9006,9006,2049785,FB,Reuters,Brazil fines Facebook $1.6 million for imprope...,https://www.investing.com/news/technology-news...
9007,9007,2050046,FB,Reuters,Gunman in Texas church had trained worshippers...,https://www.investing.com/news/world-news/gunm...
9008,9008,2049623,FB,Reuters,"Hong Kong to end year with multiple protests, ...",https://www.investing.com/news/world-news/hong...


In [ ]:
#ds_detail_news = pd.DataFrame(detail_news)
ds_detail_news.set_index("news_id")
ds_detail_news.head()
ds_detail_news.tail()

,Unnamed: 0,news_id,ticker,author,title,url
9005,9005,2049753,FB,Reuters,'Do Not Sell My Info': U.S. retailers rush to ...,https://www.investing.com/news/technology-news...
9006,9006,2049785,FB,Reuters,Brazil fines Facebook $1.6 million for imprope...,https://www.investing.com/news/technology-news...
9007,9007,2050046,FB,Reuters,Gunman in Texas church had trained worshippers...,https://www.investing.com/news/world-news/gunm...
9008,9008,2049623,FB,Reuters,"Hong Kong to end year with multiple protests, ...",https://www.investing.com/news/world-news/hong...
9009,9009,2049706,FB,Reuters,Political exercises: Thai rivals plan to mobil...,https://www.investing.com/news/world-news/poli...


In [ ]:
details = []

for news_id, news in tqdm(ds_detail_news.loc[7179:].iterrows()): 
    link = news["url"]
    news_id = news["news_id"]
    ticker = news["ticker"]
    author =news["author"]
    title = news["title"]
    wd.set_window_size(1920, 1080)
    wd.get(link)
    time.sleep(1 + np.random.randn()*0.2)
    wait = WebDriverWait(wd, 30)
    try:
      date_container = wd.find_element_by_css_selector('#leftColumn div.contentSectionDetails span').text
      if '(' in date_container:
        date_time = wd.find_element_by_css_selector('#leftColumn div.contentSectionDetails span').text.split('(')[-1].split(')')[0]
      else:
        date_time = wd.find_element_by_css_selector('#leftColumn div.contentSectionDetails span').text
      if 'Sep' in date_time:
          date = date_time.split(' ')[1].split(',')[0] + '/09/' + date_time.split(' ')[2].split(' ')[0]
      elif 'Aug' in date_time:
          date = date_time.split(' ')[1].split(',')[0] + '/08/' + date_time.split(' ')[2].split(' ')[0]
      elif 'Jul' in date_time:
          date = date_time.split(' ')[1].split(',')[0] + '/07/' + date_time.split(' ')[2].split(' ')[0]
      elif 'Jun' in date_time:
          date = date_time.split(' ')[1].split(',')[0] + '/06/' + date_time.split(' ')[2].split(' ')[0]
      elif 'May' in date_time:
          date = date_time.split(' ')[1].split(',')[0] + '/05/' + date_time.split(' ')[2].split(' ')[0]
      elif 'Apr' in date_time:
          date = date_time.split(' ')[1].split(',')[0] + '/04/' + date_time.split(' ')[2].split(' ')[0]
      elif 'Mar' in date_time:
          date = date_time.split(' ')[1].split(',')[0] + '/03/' + date_time.split(' ')[2].split(' ')[0]
      elif 'Feb' in date_time:
          date = date_time.split(' ')[1].split(',')[0] + '/02/' + date_time.split(' ')[2].split(' ')[0]
      elif 'Jan' in date_time:
          date = date_time.split(' ')[1].split(',')[0] + '/01/' + date_time.split(' ')[2].split(' ')[0]
      elif 'Dec' in date_time:
          date = date_time.split(' ')[1].split(',')[0] + '/12/' + date_time.split(' ')[2].split(' ')[0]
      elif 'Nov' in date_time:
          date = date_time.split(' ')[1].split(',')[0] + '/11/' + date_time.split(' ')[2].split(' ')[0]
      elif 'Oct' in date_time:
          date = date_time.split(' ')[1].split(',')[0] + '/10/' + date_time.split(' ')[2].split(' ')[0]
      time_stamp = date_time.split(' ')[3][0:]
      article = [] 
      container_news = wd.find_elements_by_css_selector("div.WYSIWYG.articlePage p")
      for p in range(len(container_news)):
        piece_of_article = wd.find_elements_by_css_selector("div.WYSIWYG.articlePage p")[p].text
        article.append(piece_of_article)
      details.append({'news_id': news_id,
                      'ticker': ticker,
                      'date': date,
                      'time_stamp': time_stamp,
                      'author': author,
                      'title': title,
                      'link': link,
                      'article': article
                      })
    except:
      pass
  
len(details)
ds_article_news = pd.DataFrame(details)
ds_article_news.set_index("news_id")
ds_article_news.info()
ds_article_news.to_csv('/ds_article_investing.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1831 entries, 0 to 1830
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   news_id     1831 non-null   object
 1   ticker      1831 non-null   object
 2   date        1831 non-null   object
 3   time_stamp  1831 non-null   object
 4   author      1831 non-null   object
 5   title       1831 non-null   object
 6   link        1831 non-null   object
 7   article     1831 non-null   object
dtypes: object(8)
memory usage: 114.6+ KB


In [ ]:
ds_article_news = pd.DataFrame(details)
ds_article_news.set_index("news_id")
ds_article_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   news_id     1599 non-null   object
 1   ticker      1599 non-null   object
 2   date        1599 non-null   object
 3   time_stamp  1599 non-null   object
 4   author      1599 non-null   object
 5   title       1599 non-null   object
 6   link        1599 non-null   object
 7   article     1599 non-null   object
dtypes: object(8)
memory usage: 100.1+ KB


In [ ]:
ds_article_news = pd.DataFrame(details)
ds_article_news.set_index("news_id")
ds_article_news.head(1000)

,news_id,ticker,date,time_stamp,author,title,link,article
0,2287359,AAPL,05/09/2020,10:20PM,The Motley Fool,Apple stock market cap shows just how small cr...,https://www.investing.com/news/cryptocurrency-...,[While the crypto space has seen tremendous gr...
1,2287183,AAPL,06/09/2020,08:08AM,The Motley Fool,"Weekly Comic: Apple, Tesla Slide Spark Fears O...",https://www.investing.com/news/economy/weekly-...,"[By Jesse Cohen, , Investing.com - Stocks on W..."
2,2287166,AAPL,05/09/2020,06:20AM,The Motley Fool,Epic Games asks court to prevent what it descr...,https://www.investing.com/news/technology-news...,[(Reuters) - Epic Games said late on Friday th...
3,2287028,AAPL,04/09/2020,10:55PM,The Motley Fool,Buffett's Berkshire slashes Wells Fargo stake,https://www.investing.com/news/stock-market-ne...,[(Reuters) - Warren Buffett's Berkshire Hathaw...
4,2287081,AAPL,04/09/2020,10:55PM,The Motley Fool,Why Buffett's bet on Japan could turn on highe...,https://www.investing.com/news/stock-market-ne...,"[By David Randall and Svea Herbst-Bayliss, NEW..."
...,...,...,...,...,...,...,...,...
995,2123776,AAPL,27/03/2020,12:10PM,Reuters,Apple works with White House to release 'COVID...,https://www.investing.com/news/coronavirus/app...,[(Reuters) - Apple Inc (NASDAQ:AAPL) and the W...
996,2123454,AAPL,27/03/2020,08:18AM,Reuters,Maker of Purell Hand Sanitizer Denied in Reque...,https://www.investing.com/news/stock-market-ne...,[(Bloomberg) -- With the U.S. now surpassing a...
997,2123157,AAPL,27/03/2020,03:06AM,Reuters,Olympics delay deals setback to Samsung's plan...,https://www.investing.com/news/stock-market-ne...,"[By Hyunjoo Jin, SEOUL (Reuters) - For Samsung..."
998,2122851,AAPL,26/03/2020,03:52PM,Reuters,Apple Stock Rises 4%,https://www.investing.com/news/stock-market-ne...,[Investing.com - Apple Stock rose by 4.29% to ...


In [ ]:
ds_article_news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 531 entries, 0 to 530
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   news_id     531 non-null    object
 1   ticker      531 non-null    object
 2   date        531 non-null    object
 3   time_stamp  531 non-null    object
 4   title       531 non-null    object
 5   article     531 non-null    object
 6   link        531 non-null    object
dtypes: object(7)
memory usage: 29.2+ KB


In [ ]:
ds_detail_article = pd.read_csv('/content/ds_article_investing_AAPL.csv')
ds_detail_article.head(1599)

,Unnamed: 0,news_id,ticker,date,time_stamp,author,title,link,article
0,0,2287359,AAPL,05/09/2020,10:20PM,The Motley Fool,Apple stock market cap shows just how small cr...,https://www.investing.com/news/cryptocurrency-...,['While the crypto space has seen tremendous g...
1,1,2287183,AAPL,06/09/2020,08:08AM,The Motley Fool,"Weekly Comic: Apple, Tesla Slide Spark Fears O...",https://www.investing.com/news/economy/weekly-...,"['By Jesse Cohen', '', 'Investing.com - Stocks..."
2,2,2287166,AAPL,05/09/2020,06:20AM,The Motley Fool,Epic Games asks court to prevent what it descr...,https://www.investing.com/news/technology-news...,['(Reuters) - Epic Games said late on Friday t...
3,3,2287028,AAPL,04/09/2020,10:55PM,The Motley Fool,Buffett's Berkshire slashes Wells Fargo stake,https://www.investing.com/news/stock-market-ne...,"[""(Reuters) - Warren Buffett's Berkshire Hatha..."
4,4,2287081,AAPL,04/09/2020,10:55PM,The Motley Fool,Why Buffett's bet on Japan could turn on highe...,https://www.investing.com/news/stock-market-ne...,"['By David Randall and Svea Herbst-Bayliss', ""..."
...,...,...,...,...,...,...,...,...,...
1594,1594,2037498,AAPL,10/12/2019,03:13AM,Reuters,Hong Kong cabinet reshuffle not an 'immediate ...,https://www.investing.com/news/world-news/hong...,"['By Clare Jim and Sharon Tam', 'HONG KONG (Re..."
1595,1595,2037400,AAPL,09/12/2019,05:25PM,Reuters,U.S. stocks lower at close of trade; Dow Jones...,https://www.investing.com/news/stock-market-ne...,['Investing.com – U.S. stocks were lower after...
1596,1596,2037278,AAPL,09/12/2019,05:09PM,Reuters,"Stocks - Wall Street Drops Ahead of Fed, Tarif...",https://www.investing.com/news/stock-market-ne...,"['', 'Investing.com - Stocks moved lower on Mo..."
1597,1597,2037266,AAPL,09/12/2019,03:39PM,Reuters,"Taiwan Stocks Will Soar to Record in 2020, Ana...",https://www.investing.com/news/stock-market-ne...,['(Bloomberg) -- This year’s unexpected 20% ra...
